In [1]:
from dotenv import load_dotenv
load_dotenv()

# get to working directory of llms
import sys
sys.path.append('/pub/anvieyra/copilot/llms/user_modules')
from db import Session, get_training_data, add_sentiments_data
from helper_functions import generated_decoded_output, find_sentiment
import pandas as pd

In [2]:
# get training data for sentiment analysis
training_data = get_training_data()
training_data.head()

,dialogue_id,dataset,dialogue_text,actual_summary,actual_sentiment
0,3816,training,"#Person1#: Good morning, sir. Is there anythin...",#Person2# wants a suit made in 100 % black Bar...,neutral
1,3,training,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,positive
2,4,training,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,negative
3,5,training,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,positive
4,6,training,"#Person1#: Happy birthday, Aims!\n#Person2#: T...",Lisa gives Aims a birthday present and Aims lo...,positive


In [3]:
unproc_deci = training_data.apply(generated_decoded_output, axis = 1, model_id = "Deci/DeciLM-7B-instruct")
unproc_deci.head()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

,dialogue_id,dataset,dialogue_text,actual_summary,actual_sentiment,generated_sentiment,memory_sentiment_usage,time_sentiment_taken
0,3816,training,"#Person1#: Good morning, sir. Is there anythin...",#Person2# wants a suit made in 100 % black Bar...,neutral,"Given the following dialogue, output a single ...",27177.072266,0.348379
1,3,training,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,positive,"Given the following dialogue, output a single ...",27200.726074,0.281281
2,4,training,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,negative,"Given the following dialogue, output a single ...",27200.142090,0.280755
3,5,training,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,positive,"Given the following dialogue, output a single ...",27197.362793,0.283687
4,6,training,"#Person1#: Happy birthday, Aims!\n#Person2#: T...",Lisa gives Aims a birthday present and Aims lo...,positive,"Given the following dialogue, output a single ...",27177.364258,0.213981


In [4]:
unproc_deci.to_csv("deci-sentiment.csv")

In [5]:
proc_deci = unproc_deci.apply(find_sentiment, axis = 1)

In [6]:
proc_deci["model_id"] = "deciLM 7b"

In [7]:
proc_deci = proc_deci[["dialogue_id", "model_id", "generated_sentiment", "memory_sentiment_usage", "time_sentiment_taken"]]

In [8]:
add_sentiments_data(proc_deci)